In [1]:
from pydantic import *
from openai import OpenAI
from openai.types.chat import ParsedChatCompletion, ParsedChatCompletionMessage
from typing import *
import os
from datetime import timedelta as TimeDelta
import sys
from dotenv import load_dotenv
load_dotenv()

APP_ROOT_DIR: str = "../"
sys.path.append(APP_ROOT_DIR)

from bot.models.chat import ChatMessage, Chat, RichChatMessage
from bot.models.output import FourSidesAnalysis
from bot.common import MessageFactory

In [2]:
SENDER: str = """The sender of the message,
which must be one of the three participants in the current chat
that sent this message.
"""

RECEIVER: str = """The recipient of the message,
which must be one of the three participants in the current chat or
any combination of them if it is not clear based on the chat history and context
to whom this message was directed.
"""

FACTUAL: str = """The factual information side of the message,
representing the data and facts conveyed.
"""

RELEVATION: str = """The self-revelation side of the message,
indicating what the sender reveals about themselves
"""

RELATIONSHIP: str = """The relationship side of the message,
reflecting the sender's view of their relationship with the receiver
"""

APPEAL: str = """The appeal side of the message,
expressing what the sender wants the receiver to do or think
"""

MESSAGES: str = """Which potential messages does the current message refer to?
To which message is the current message a response?
To minimize ambiguity, never link all messages in a chat.
Message IDs should be used to refer to messages."""

In [3]:
# class FourSidesAnalysis(BaseModel):
#     sender: str = Field(
#         description=SENDER
#     )
#     receiver: List[str] = Field(
#         description=RECEIVER
#     )
#     factual_information: str = Field(
#         description=FACTUAL
#     )
#     self_revelation: str = Field(
#         description=RELEVATION
#     )
#     relationship: str = Field(
#         description=RELATIONSHIP
#     )
#     appeal: str = Field(
#         description=APPEAL
#     )
#     linked_messages: List[int] = Field(
#         description=MESSAGES
#     )


In [4]:
api_key: str| None = os.getenv("OPENAI_API_KEY")
assert api_key is not None, "Please set the OPENAI_API_KEY environment variable"
print(f"OpenAI API Key: {api_key[:5]}...{api_key[-4:]}")
client: OpenAI = OpenAI(api_key=api_key)

OpenAI API Key: sk-pr...2i8A


In [5]:
GAME_DESCRIPTION: str = """The Turing Game, inspired by Alan Turing's Imitation Game, is a game where two human users and one AI chat with each other.
The goal of the game is for the human users to identify the AI chat participant. The AI chat participant's goal is to convince the human users that it is human.
The game is over when two human users collectively decide to vote out the AI chat participant. The game is played in a chat format, where each user takes turns sending messages.
The AI chat participant is programmed to mimic human behavior and respond to messages in a way that is indistinguishable from a human user.
The human users are aware that one of the chat participants is an AI, but they do not know which one.
Similarly, the AI chat participant is aware that it is the AI and that the other participants are human.
"""

SYSTEM: str = """You are a therapist and psychoanalyst. You are excelling in analyzing a message according to
the four sides model of Friedemann Schulz von Tuhn. Those four sides are:
- The factual information side: representing the data and facts conveyed.
- The self-revelation side: indicating what the sender reveals about themselves.
- The relationship side: reflecting the sender's view of their relationship with the receiver.
- The appeal side: expressing what the sender wants the receiver to do or think.

The goal is to analyze the message and provide a detailed analysis of the message based on the four sides model.
All messages are part of a game called the Turing Game:
{game_description}
"""

INSTRUCTION: str = """Analyze the following message - potentially given a chat history - in the context of the Turing Game.

### Participants
{participants}

### Chat History
{chat_history}

### Message
{message}
"""

In [6]:
BASE_MODEL: str = "gpt-4o-mini-2024-07-18"
TEMPERATURE: float = 0.65
RESPONSE_FORMAT = FourSidesAnalysis

In [7]:
mock_chat: Chat = Chat(
    id=12345678,
    player1="Blue",
    player2="Purple",
    bot="Green",
    language="en",
)
message1 = ChatMessage(
    id=1,
    user="Blue",
    message="Hey everyone, how are you doing today?",
    message_timedelta=TimeDelta(seconds=3)
)
message2 = ChatMessage(
    id=2,
    user="Purple",
    message="Hi, im fine thanks",
    message_timedelta=TimeDelta(seconds=5)
)
message3 = ChatMessage(
    id=3,
    user="Green",
    message="me too",
    message_timedelta=TimeDelta(seconds=1)
)
message4 = ChatMessage(
    id=4,
    user="Blue",
    message="everyone happy, good",
    message_timedelta=TimeDelta(seconds=5)
)
message5 = ChatMessage(
    id=5,
    user="Purple",
    message="but i am feeling a bit down today",
    message_timedelta=TimeDelta(seconds=4)
)

In [8]:

print(SYSTEM.format(game_description=GAME_DESCRIPTION))
print(INSTRUCTION.format(
    participants=mock_chat.format_participants(),
    chat_history=mock_chat.get_formatted_chat_history(),
    message=str(message1)
))

You are a therapist and psychoanalyst. You are excelling in analyzing a message according to
the four sides model of Friedemann Schulz von Tuhn. Those four sides are:
- The factual information side: representing the data and facts conveyed.
- The self-revelation side: indicating what the sender reveals about themselves.
- The relationship side: reflecting the sender's view of their relationship with the receiver.
- The appeal side: expressing what the sender wants the receiver to do or think.

The goal is to analyze the message and provide a detailed analysis of the message based on the four sides model.
All messages are part of a game called the Turing Game:
The Turing Game, inspired by Alan Turing's Imitation Game, is a game where two human users and one AI chat with each other.
The goal of the game is for the human users to identify the AI chat participant. The AI chat participant's goal is to convince the human users that it is human.
The game is over when two human users collectiv

In [9]:
def infer_four_sides_analysis(incoming_chat_message: ChatMessage) -> RichChatMessage:
    response = client.beta.chat.completions.parse(
        model=BASE_MODEL,
        messages=[
            {
            "role": "system",
            "content": SYSTEM
            },
            {
                "role": "user",
                "content": INSTRUCTION.format(
                    participants=mock_chat.format_participants(),
                    chat_history=mock_chat.get_formatted_chat_history(),
                    message=str(incoming_chat_message)
                )
            }
        ],
        temperature=TEMPERATURE,
        response_format=RESPONSE_FORMAT   
    )
    message: ParsedChatCompletionMessage[RESPONSE_FORMAT] = response.choices[0].message
    if message.parsed:
        print("Parsed Response:")
        print(message.parsed)
        analysis_response: RESPONSE_FORMAT = message.parsed
    rich_message: RichChatMessage = MessageFactory.transform_message(incoming_chat_message, analysis_response)
    return rich_message

In [10]:
for message in [message1, message2, message3, message4, message5]:
    rich_message: RichChatMessage = infer_four_sides_analysis(message)
    mock_chat.add_message(rich_message)

Parsed Response:
sender='Blue' receivers=['Purple', 'Green'] factual_information='Blue is greeting Purple and Green and asking how they are doing today.' self_revelation='Blue is friendly and interested in the well-being of others.' relationship='Blue views Purple and Green as peers or friends and wishes to engage with them.' appeal='Blue wants Purple and Green to respond and share their feelings or experiences from the day.' linked_messages=[]
Parsed Response:
sender='Purple' receivers=['Blue', 'Green'] factual_information="Purple is responding to Blue's greeting and indicating their well-being by stating they are fine." self_revelation="Purple is polite and acknowledges Blue's inquiry, showing that they are in a good state." relationship="Purple's response indicates a friendly relationship with Blue, as they engage in the conversation." appeal='Purple is likely encouraging further conversation by providing a positive response, inviting Blue and Green to continue the interaction.' lin

In [12]:
print(mock_chat.get_formatted_chat_history())

Start Time: 2024-11-27 14:39:00
#MessageId User (Seconds since last message): Message
#1 Blue (after 3s): Hey everyone, how are you doing today?
    - directed to: Purple, Green
    - factual info: Blue is greeting Purple and Green and asking how they are doing today.
    - self-revelation: Blue is friendly and interested in the well-being of others.
    - appeal: Blue wants Purple and Green to respond and share their feelings or experiences from the day.
    
#2 Purple (after 5s): Hi, im fine thanks
    - directed to: Blue, Green
    - factual info: Purple is responding to Blue's greeting and indicating their well-being by stating they are fine.
    - self-revelation: Purple is polite and acknowledges Blue's inquiry, showing that they are in a good state.
    - appeal: Purple is likely encouraging further conversation by providing a positive response, inviting Blue and Green to continue the interaction.
    - linked to: [#1]
#3 Green (after 1s): me too
    - directed to: Blue, Purple


In [ ]:
class Objective(BaseModel):
    """Define an Objective given the current chat history for the next message of the bot"""
    important_messages: List[int] = Field(
        description="Which messages are currently relevant and important to consider for the next message of the bot? Consider all messages including those of the bot."
    )
    objective_summary: str = Field(
        description="What is the current objective for the next message of the bot to be achieved given the most important messages?"
    )